In [1]:
import reasoning_gym
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from trl import GRPOTrainer
import torch

/home/albatrosfirst/rlhf_finetuning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = reasoning_gym.create_dataset('gcd', size=1000,seed=42)

In [3]:
for i, x in enumerate(dataset):
    print(f"q:{x['question']}\na:{x['answer']}\nmetadata: {x['metadata']}\n")

q:Find the Greatest Common Divisor (GCD) of these numbers: 26, 760. Give only the GCD as your final answer.
a:2
metadata: {'source_dataset': 'gcd', 'source_index': 0, 'numbers': [26, 760], 'result': 2, 'num_terms': 2, 'difficulty': {'num_terms': (2, 2), 'value': (1, 1000)}}

q:Find the Greatest Common Divisor (GCD) of these numbers: 688, 716. Give only the GCD as your final answer.
a:4
metadata: {'source_dataset': 'gcd', 'source_index': 1, 'numbers': [688, 716], 'result': 4, 'num_terms': 2, 'difficulty': {'num_terms': (2, 2), 'value': (1, 1000)}}

q:Find the Greatest Common Divisor (GCD) of these numbers: 297, 30. Give only the GCD as your final answer.
a:3
metadata: {'source_dataset': 'gcd', 'source_index': 2, 'numbers': [297, 30], 'result': 3, 'num_terms': 2, 'difficulty': {'num_terms': (2, 2), 'value': (1, 1000)}}

q:Find the Greatest Common Divisor (GCD) of these numbers: 428, 500. Give only the GCD as your final answer.
a:4
metadata: {'source_dataset': 'gcd', 'source_index': 3, 'n

In [5]:
model_id = "Qwen/Qwen3-0.6B-Base"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

In [ ]:
system_prompt = """
Generate a answer after reasoning:
<resoning>
display your reasons here 
</resonning>

<answer>
answer here
</answer>

You must answer here <answer>...</answer>
"""

In [ ]:
pipe = pipeline("text-generation", model=model_id)
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": dataset[0]['question']},
]

inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))